In [16]:
import pandas as pd
import numpy as np

In [2]:
url = "results-gpt4o-mini.csv"

In [3]:
df = pd.read_csv(url)
df = df.iloc[:300]

In [4]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


### Q1. Getting the embeddings model

In [6]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

/Users/kristinakhvatova/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/Users/kristinakhvatova/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
### create the embeddings for the first LLM answer
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [10]:
embedding = embedding_model.encode(answer_llm)
embedding[0]

-0.42244682

### Q2. Computing the dot product

In [13]:
embedded_answer_llm = []
emdedded_answer_orig = []
dot_product_list = []

for i in range(0, len(df)):
    
    embedding_llm = embedding_model.encode(df["answer_llm"][i])
    embedding_orig = embedding_model.encode(df["answer_orig"][i])
    
    dot_product = embedding_llm.dot(embedding_orig)
    
    embedded_answer_llm.append(embedding_llm)
    emdedded_answer_orig.append(embedding_orig)
    dot_product_list.append(dot_product)
    
df['embedded_answer_llm'] = embedded_answer_llm
df['emdedded_answer_orig'] = emdedded_answer_orig
df['dot_product'] = dot_product_list

df.head()

,answer_llm,answer_orig,document,question,course,embedded_answer_llm,emdedded_answer_orig,dot_product
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244682, -0.22485572, -0.32405868, -0.284...","[-0.0302141, -0.34443897, -0.28076288, 0.06150...",17.515995
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068154, 0.047848746, -0.31510997, -0.210...","[-0.0302141, -0.34443897, -0.28076288, 0.06150...",13.418407
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.05881398, -0.3373695, -0.36157578, 0.02173...","[-0.0302141, -0.34443897, -0.28076288, 0.06150...",25.313251
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753678, -0.008134156, -0.21719904, -0.11...","[-0.0302141, -0.34443897, -0.28076288, 0.06150...",12.147418
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969349, -0.5005089, -0.16598417, 0.30666...","[-0.0302141, -0.34443897, -0.28076288, 0.06150...",18.747723


In [15]:
df.describe()

,dot_product
count,300.000000
mean,27.495996
std,6.384743
min,4.547927
25%,24.307843
50%,28.336861
75%,31.674304
max,39.476013


### Q3. Computing the cosine

In [19]:
def normalize_array(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

def dot_product(row):
    return np.dot(row['embedded_answer_llm_norm'], row['emdedded_answer_orig_norm'])

In [20]:
df['embedded_answer_llm_norm'] = df['embedded_answer_llm'].apply(normalize_array)
df['emdedded_answer_orig_norm'] = df['emdedded_answer_orig'].apply(normalize_array)

df['dot_product_norm'] = df.apply(dot_product, axis=1)

In [22]:
df.describe()

,dot_product,dot_product_norm
count,300.000000,300.000000
mean,27.495996,0.728392
std,6.384743,0.157755
min,4.547927,0.125357
25%,24.307843,0.651273
50%,28.336861,0.763761
75%,31.674304,0.836235
max,39.476013,0.958796


### Q4. Rouge

In [24]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata


In [25]:
from rouge import Rouge
rouge_scorer = Rouge()

In [27]:
df['answer_llm'].loc[10], df['answer_orig'].loc[10]

("Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.",
 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.')

In [29]:
scores = rouge_scorer.get_scores(df['answer_llm'].loc[10], df['answer_orig'].loc[10])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

### Q5. Average rouge score

In [37]:
np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])

0.35490034990035496

### Q6. Average rouge score for all the data points

In [38]:
def rouge_score(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return scores['rouge-2']['f']

In [39]:
df['rouge_2_f'] = df.apply(rouge_score, axis=1)

In [42]:
np.mean(df['rouge_2_f'])

0.20696501983423318